In [5]:
import pandas
import matplotlib.pyplot as plt

import sys
sys.path.append('/home/jshe/prop-pred/src/data')
from utils import *

import torch
import torch.nn as nn
import torch_geometric as torchg
import torch_geometric.nn as gnn
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_smiles

from model import *

from rdkit.Chem import MolFromSmiles

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [6]:
datadir = '../../data/regress_immuno/'
data = to_namedtuple(to_tensor(load_data(datadir), device))

In [7]:
train_X = data.train.smiles
validation_X = data.validation.smiles
test_X = data.test.smiles

In [149]:
train_Y = data.train.y
validation_Y = data.validation.y
test_Y = data.test.y

In [15]:
train_X[100:116]

array(['COC(=O)CNC(c1ccccc1)c1cc(Br)ccc1NC(=O)c1ccccc1Cl',
       'O=C(NCC1CCCO1)c1[nH]nnc1-c1ccc(F)cc1',
       'O=C1NC(=O)C(=CNc2cccc(O)c2)C(=O)N1',
       'N#Cc1c(Oc2ccc(F)c(NC(=O)Cc3cccc(C(F)(F)F)c3)c2)ccc2nc(NC(=O)C3CC3)sc12',
       'C=CC(=O)N1CC(Nc2ncnc3[nH]ccc23)CCC1C',
       'Cc1nnc(NN=Cc2ccc(Cl)c([N+](=O)[O-])c2)[nH]c1=O',
       'Cc1cccc(CN2CCN(C(c3ccccc3)c3ccc(Cl)cc3)CC2)c1.Cl.Cl',
       'O=c1c(O)c(-c2ccc(O)cc2O)oc2cc(O)cc(O)c12',
       'CCOc1ccc(C=C2SC(=S)N(C)C2=O)cc1', 'COc1ccccc1OCC(O)COC(N)=O',
       'O=S(=O)(c1ccc(F)cc1)N1CCC(c2nc3ccccc3[nH]2)CC1',
       'NC1(C(=O)NC(CCO)c2ccc(Cl)cc2)CCN(c2ncnc3[nH]ccc23)CC1',
       'Nc1nc(NC2CC2)c2ncn(C3C=CC(CO)C3)c2n1',
       'O=C(O)c1ccc2c3c1cccc3c(=O)n1c3ccccc3nc21',
       'CCN1C(C)=C(C(=O)OC)C(c2ccc([N+](=O)[O-])cc2)C(C(=O)OC)=C1C',
       'CCN1/C(=C/C(C)=O)Sc2ccc(OC)cc21'], dtype=object)

In [150]:
train_graphs = [from_smiles(smiles) for smiles in data.train.smiles]

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=False)

## Model

In [159]:
model = Transformer(
    in_features=9, embed_channels=64, 
    heads=4, out_features=1, dropout=0.2
).to(device)

In [160]:
x = next(iter(train_loader)).to(device)

In [161]:
y = model(x)

In [163]:
y.shape

torch.Size([891, 16])